![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Extending the internal databases with audio features

At this point, you have the **hot_songs** and the **not_hot_songs** databases. However, you don't have any acoustic information about the songs. 
The purpose of this lab is to use Spotify's API to extend both databases with this information for each song included to use this information later.

# Instructions

* Create a function to search a given **single** song in the Spotify API: **search_song(title, artist)**. 

Take into account that sometimes Spotify's API will return several matches for the same song title (different artists, a different album of the same artist, version of the song,...etc). Then it will be nice to display a list of outputs to the user and let him/her select which is the right match. Once the desired song is located, the function should return the href/id/uri of the song to the code (not to the user).
Keep in mind, that a given song might not be available on Spotify's API (make sure to use the song's title and artist searching the song). If the song is not found, the function must return an empty string as the href/id/uri. Also, in this case, you should remove this song from the database.

* Create a function **get_audio_features(list_of_song_ids)** to obtain the audio features of a given list of songs (the content of list_of_songs can be the href/id/uri). 

Be careful to not exceed the number of calls to the API otherwise, you will be banned and you will have to wait several hours before launching a new request [see here](https://developer.spotify.com/documentation/web-api/guides/rate-limits/).
A good strategy to prevent this problem is to split the list of song id's in "chunks" of 50 songs id's and wait 20 seconds before asking for the audio features of the next "chunk".
Then, use this function to create a Pandas Dataframe with the audio features of the list of songs. Hint: create a dictionary with the song's audio features as keys and an **empty list as values**. 
Then fill in the lists with the corresponding audio features of each song. Finally, create your data frame from the dictionary.

* Once the previous function has been created, create another function **add_audio_features(df, audio_features_df)** to concat a given dataframe with the dataframe containing the audio features alongside any other desired info, and return the extended data frame.
Replace the old internal files of songs (hot and not hot) with the extended data frames with the audio features and save them into separate files on the disk.

# 0 Import libraries

In [23]:
import random
import time
import pprint
import pandas as pd
import spotipy
import json
from IPython.display import clear_output
from spotipy.oauth2 import SpotifyClientCredentials

from credentials import *
import functions


# 1 Load Datasets

In [24]:
# Get the hot 100 dataset
hot_df = functions.scrape_hot100()

# Show top 5 rows
display(hot_df.head())

,title,artist
0,Last Night,Morgan Wallen
1,Fast Car,Luke Combs
2,Vampire,Olivia Rodrigo
3,Calm Down,Rema & Selena Gomez
4,I Can See You (Taylor's Version) (From The Vault),Taylor Swift


In [25]:
# Get the not hot dataset
not_hot_df = pd.read_csv("million_song_subset.csv", sep = ";")

# Show top 5 rows
display(not_hot_df.head())

,title,artist
0,Je Sais Que La Terre Est Plate,Rapha\xc3\xal
1,On Efface,Julie Zenatti
2,Howells Delight,The Baltimore Consort
3,Martha Served,I Hate Sally
4,Zip-A-Dee-Doo-Dah (Song of the South),Orlando Pops Orchestra


# 2 Concat Hot and Not Hot Datasets

In [26]:
# Add a column to each df to determine if it is hot or not
not_hot_df['is_hot'] = False
hot_df['is_hot'] = True

In [27]:
# Concat vertically both dataframes. Drop the old index and create new one.
all_songs_df = pd.concat([not_hot_df, hot_df]).reset_index(drop=True)

# Create a test df subset to develop functions
test_df = all_songs_df.sample(n=20, random_state=42)

# Show concat df
all_songs_df.head()

,title,artist,is_hot
0,Je Sais Que La Terre Est Plate,Rapha\xc3\xal,False
1,On Efface,Julie Zenatti,False
2,Howells Delight,The Baltimore Consort,False
3,Martha Served,I Hate Sally,False
4,Zip-A-Dee-Doo-Dah (Song of the South),Orlando Pops Orchestra,False


# 3 Search Function

In [28]:
# Create a function to search a given single song in the Spotify API: search_song(title, artist).

def search_song(title: str, artist: str) -> list:
    '''
    Given a song title and artist, this function returns up to 5 posible matches
    from Spotipy, Spotify's API.
    
    Input:
    title: song title as string
    artist: song artist as string
    
    Output:
    List of matches, each match is a dictionary
    
    '''
    
    # Initialize SpotiPy with user credentias
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID,
                                                           client_secret=Client_Secret))

    # Create the query string
    query = ("track:"+title if title != "" else "")+" "+("artist:"+artist if artist != "" else "")

    # Search the requested song and artist if the input is not blank
    if len(query) > 0:
        #try:
            raw_results = sp.search(q=query,type="track,artist",limit=5)
            raw_results = raw_results['tracks']['items'] # Parse first two dictionary levels
        #except Exception as e:
            #print("Error:", e)
    else:
        raw_results = []
    
    # Define result variable
    clean_results = []
    
    # Loop through results and store relevant data in clean_results as list of dictionaries
    for index, result in enumerate(raw_results):
        
        # Store each individual result in temporal dictionary             TO-DO: add -> song poopularity, artist nationality...
        clean_result = {'result_index': index,
                        'track_id': result['id'],
                        'href': result['href'],
                        'track_link': result['external_urls']['spotify'],
                        'track_name': result['name'],
                        'artist': result['album']['artists'][0]['name'],
                        'album_name': result['album']['name'],
                        'album_release_year': result['album']['release_date'],
                        'track_duration': round(result['duration_ms']/1000),
                        'album_image': result['album']['images'][1]['url'],
                        'popularity': result['popularity'],
        }
        
        # Append each dictionary to results list
        clean_results.append(clean_result)
    
    return clean_results



In [29]:
# Test function
search_song('Scar Tissue', 'Red Hot')

[{'result_index': 0,
  'track_id': '1G391cbiT3v3Cywg8T7DM1',
  'href': 'https://api.spotify.com/v1/tracks/1G391cbiT3v3Cywg8T7DM1',
  'track_link': 'https://open.spotify.com/track/1G391cbiT3v3Cywg8T7DM1',
  'track_name': 'Scar Tissue',
  'artist': 'Red Hot Chili Peppers',
  'album_name': 'Californication (Deluxe Edition)',
  'album_release_year': '1999-06-08',
  'track_duration': 216,
  'album_image': 'https://i.scdn.co/image/ab67616d00001e0294d08ab63e57b0cae74e8595',
  'popularity': 85},
 {'result_index': 1,
  'track_id': '6UYet0CM4Dz0emx44RTQbc',
  'href': 'https://api.spotify.com/v1/tracks/6UYet0CM4Dz0emx44RTQbc',
  'track_link': 'https://open.spotify.com/track/6UYet0CM4Dz0emx44RTQbc',
  'track_name': 'Scar Tissue',
  'artist': 'Various Artists',
  'album_name': 'Musica Soft per Smart Working',
  'album_release_year': '2023-07-21',
  'track_duration': 216,
  'album_image': 'https://i.scdn.co/image/ab67616d00001e025683719c3fab6baec2921526',
  'popularity': 0},
 {'result_index': 2,
  '

# 3 Get Data Function

In [30]:
# Create a function to get all the Spotify song data

def get_song_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Given a DataFrame with two columns: title and artist,
    this function returns a new df with track data 
    using Spotipy, Spotify's API.
    
    Input:
    df: pandas DataFrame with songs
    
    Output:
    DataFrame with the track data
    
    '''
    
    # Define loop pauses to avoid API server blockage by overflow
    sleep_time = 30 # seconds of sleep time
    loop_count = 50 # tracks per sleep loop
    
    # Define an empty array to append results
    sp_songs = []
    
    loop_counter = 0 # counter to track loop
    counter = 0 # counter to track progress
    fail_counter = 0 # counter to track failed matches
                 
    for index, row in df.iterrows():
        
        counter += 1 # Add one to the global song counter
        
        # Clear last print and print next progress
        clear_output(wait=True)
        print('Downloading song Data...', round(((counter + 1) / df.shape[0])*100), '%')
        
        # Get the current row song title and artist. Fix to maximum 30 chars to avoid API error.
        song_name = str(row['title'])[:30]
        artist_name = str(row['artist'])[:30]
        
        try:
            sp_data = search_song(song_name, artist_name) # Call search_song function to get data
            if len(sp_data) > 0:
                sp_data[0]['is_hot'] = row['is_hot'] # Add is_hot key-value
                sp_songs.append(sp_data[0]) # If there is data, append to global variable
            else:
                fail_counter += 1 # If there is no data, add 1 to fail_counter
        except Exception as e:
            # Log error and add counter
            print("Error:", e)
            fail_counter += 1
        
        # Sleep to avoid API overflow...
        loop_counter += 1
        if loop_counter >= loop_count:
            loop_counter = 0
            print('Sleeping for', sleep_time, 's...') # Print sleep time
            time.sleep(sleep_time) # Sleep -> pause download to simulate human behavior

    # Print download summary
    clear_output(wait=True)
    print("Song data download complete.", "Success rate:", round(((counter-fail_counter)/counter)*100), "%")
    print("Succesful downloads:",counter-fail_counter,"Failed downloads:",fail_counter)
    
    # Convert list of dictionaries to DataFrame
    sp_songs_df = pd.DataFrame(sp_songs)
    
    # Drop result_index column: used for single songs searches in search_song() function
    sp_songs_df.drop(inplace=True, columns='result_index')
    
    return sp_songs_df

In [9]:
# Test function
sp_songs_df = get_song_data(test_df)

# Show top 5 rows
sp_songs_df.head()

Song data download complete. Success rate: 35 %
Succesful downloads: 7 Failed downloads: 13


,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image,popularity,is_hot
0,1JUiLgTzZBJj5JXXvgnYn1,https://api.spotify.com/v1/tracks/1JUiLgTzZBJj...,https://open.spotify.com/track/1JUiLgTzZBJj5JX...,Out In The Woods,Leon Russell,Carney,1972,218,https://i.scdn.co/image/ab67616d00001e0251247a...,43,False
1,1bjgferMWzZ3LA9ZtRakJU,https://api.spotify.com/v1/tracks/1bjgferMWzZ3...,https://open.spotify.com/track/1bjgferMWzZ3LA9...,Can You Hear The Angels,Various Artists,"Verity Records: The First Decade, A Celebratio...",2005-09-13,224,https://i.scdn.co/image/ab67616d00001e024bccc2...,0,False
2,2c6RnmhfQQIgXU1bgzrwV9,https://api.spotify.com/v1/tracks/2c6RnmhfQQIg...,https://open.spotify.com/track/2c6RnmhfQQIgXU1...,A Distant Voice,Sonny Simmons,Staying on the Watch,2010-04-27,443,https://i.scdn.co/image/ab67616d00001e0275aa3a...,2,False
3,4BjRnuxgtWCZdTpjPUHYf6,https://api.spotify.com/v1/tracks/4BjRnuxgtWCZ...,https://open.spotify.com/track/4BjRnuxgtWCZdTp...,Hardcore From The Underland,Various Artists,Best Of Dwarf Records,2010-01-11,683,https://i.scdn.co/image/ab67616d00001e02a320ef...,8,False
4,5OmumaYiHYXUsYZPGLFTYJ,https://api.spotify.com/v1/tracks/5OmumaYiHYXU...,https://open.spotify.com/track/5OmumaYiHYXUsYZ...,Between Love & Hate,The Strokes,Room On Fire,2003-10-28,197,https://i.scdn.co/image/ab67616d00001e020f3572...,55,False


# 4 Get Audio Features Function

+ [Description of Audio Features](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)

In [15]:
# Create a function to search a given single song in the Spotify API: search_song(title, artist).

def get_audio_features(track_ids: list) -> pd.DataFrame:
    '''
    Given a list of track IDs, this function returns 
    track audio features such as danceability, energy or tempo
    using Spotipy, Spotify's API.
    
    Input:
    track_ids: list of Spotipy track_ids
    
    Output:
    DataFrame with the track_ids and audio features
    
    '''
    
    # Define loop pauses to avoid API server blockage by overflow
    sleep_time = 30 # seconds of sleep time
    loop_count = 50 # tracks per sleep loop
    
    # Initialize SpotiPy with user credentias
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID,
                                                           client_secret=Client_Secret))

    # Define result variable
    clean_results = []
    
    loop_counter = 0 # counter to track loop
    counter = 0 # counter to track progress
    fail_counter = 0 # counter to track failed matches
    
    # Loop through track_ids to get relevant audio features
    for index, track_id in enumerate(track_ids):
        
        counter += 1 # Add 1 to global counter
        clear_output(wait=True) # Clear print output to update progress
        print('Downloading audio features...', round(((index + 1) / len(track_ids))*100), '%') # Print progress
        
        try:
            # Get audio features from Spotipy
            api_result = sp.audio_features(track_id)[0]

            # Add features to track variable
            track = {'track_id': track_id,
                     'acousticness': api_result['acousticness'],
                     'danceability': api_result['danceability'],
                     'energy': api_result['energy'],
                     'instrumentalness': api_result['instrumentalness'],
                     'key': api_result['key'],
                     'liveness': api_result['liveness'],
                     'loudness': api_result['loudness'],
                     'mode': api_result['mode'],
                     'speechiness': api_result['speechiness'],
                     'tempo': api_result['tempo'],
                     'time_signature': api_result['time_signature'],
                     'valence': api_result['valence'],
            }

            # Append track results to results list
            clean_results.append(track)

        except Exception as e:
            # If there is an error:
            print("Error:", e)
            fail_counter += 1
        
        # Sleep to avoid API overflow...
        loop_counter += 1
        if loop_counter >= loop_count:
            loop_counter = 0
            print('Sleeping for', sleep_time, 's...')
            time.sleep(sleep_time) # Sleep
    
    clear_output(wait=True)
    print("Audio features download complete.", "Success rate:", round(((counter-fail_counter)/counter)*100), "%")
    print("Succesful downloads:",counter-fail_counter,"Failed downloads:",fail_counter)
    
    # Convert list of dictionaries to DataFrame
    clean_results_df = pd.DataFrame(clean_results)

    return clean_results_df



In [16]:
# Create list of track_ids from the Spotify song dataset
track_ids = list(sp_songs_df['track_id'])
track_ids

['1JUiLgTzZBJj5JXXvgnYn1',
 '1bjgferMWzZ3LA9ZtRakJU',
 '2c6RnmhfQQIgXU1bgzrwV9',
 '4BjRnuxgtWCZdTpjPUHYf6',
 '5OmumaYiHYXUsYZPGLFTYJ',
 '2CjP29VvviVwtUcTDUycHi',
 '63GYPSzNsUdAROhzeferhG']

In [17]:
# Test function
audio_features_df = get_audio_features(track_ids)

# Show results
audio_features_df.head()

Audio features download complete. Success rate: 100 %
Succesful downloads: 7 Failed downloads: 0


,track_id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1JUiLgTzZBJj5JXXvgnYn1,0.179000,0.492,0.5250,0.000000,9,0.0642,-14.665,1,0.0335,159.952,4,0.9370
1,1bjgferMWzZ3LA9ZtRakJU,0.637000,0.455,0.5740,0.003750,5,0.1370,-9.911,1,0.0451,164.827,4,0.6530
2,2c6RnmhfQQIgXU1bgzrwV9,0.824000,0.241,0.0112,0.105000,0,0.0686,-17.779,1,0.0424,72.011,5,0.0782
3,4BjRnuxgtWCZdTpjPUHYf6,0.003120,0.477,0.9530,0.000896,8,0.4240,-4.317,1,0.1030,189.096,4,0.1660
4,5OmumaYiHYXUsYZPGLFTYJ,0.000063,0.455,0.6030,0.530000,9,0.0966,-4.919,1,0.0255,104.179,4,0.5390


# 5 Concat Function

In [18]:
# Create function to concatenate dataframes

def add_audio_features(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    '''
    This function concatenates the audio features columns to the base 
    song DataFrame columns and removes any duplicate columns of the 
    concatenates DataFrame.
    
    Input:
    df: base songs DataFrame
    audio_features
    
    '''
    
    # Merge datasets based on track_id column
    merged_df = pd.merge(df1, df2, on='track_id', how='inner')
    
    # Remove duplicate columns
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    
    return merged_df



In [19]:
# Test function
concat_df = add_audio_features(sp_songs_df, audio_features_df)

# Show concat dataset
concat_df.head()

,track_id,href,track_link,track_name,artist,album_name,album_release_year,track_duration,album_image,popularity,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1JUiLgTzZBJj5JXXvgnYn1,https://api.spotify.com/v1/tracks/1JUiLgTzZBJj...,https://open.spotify.com/track/1JUiLgTzZBJj5JX...,Out In The Woods,Leon Russell,Carney,1972,218,https://i.scdn.co/image/ab67616d00001e0251247a...,43,...,0.5250,0.000000,9,0.0642,-14.665,1,0.0335,159.952,4,0.9370
1,1bjgferMWzZ3LA9ZtRakJU,https://api.spotify.com/v1/tracks/1bjgferMWzZ3...,https://open.spotify.com/track/1bjgferMWzZ3LA9...,Can You Hear The Angels,Various Artists,"Verity Records: The First Decade, A Celebratio...",2005-09-13,224,https://i.scdn.co/image/ab67616d00001e024bccc2...,0,...,0.5740,0.003750,5,0.1370,-9.911,1,0.0451,164.827,4,0.6530
2,2c6RnmhfQQIgXU1bgzrwV9,https://api.spotify.com/v1/tracks/2c6RnmhfQQIg...,https://open.spotify.com/track/2c6RnmhfQQIgXU1...,A Distant Voice,Sonny Simmons,Staying on the Watch,2010-04-27,443,https://i.scdn.co/image/ab67616d00001e0275aa3a...,2,...,0.0112,0.105000,0,0.0686,-17.779,1,0.0424,72.011,5,0.0782
3,4BjRnuxgtWCZdTpjPUHYf6,https://api.spotify.com/v1/tracks/4BjRnuxgtWCZ...,https://open.spotify.com/track/4BjRnuxgtWCZdTp...,Hardcore From The Underland,Various Artists,Best Of Dwarf Records,2010-01-11,683,https://i.scdn.co/image/ab67616d00001e02a320ef...,8,...,0.9530,0.000896,8,0.4240,-4.317,1,0.1030,189.096,4,0.1660
4,5OmumaYiHYXUsYZPGLFTYJ,https://api.spotify.com/v1/tracks/5OmumaYiHYXU...,https://open.spotify.com/track/5OmumaYiHYXUsYZ...,Between Love & Hate,The Strokes,Room On Fire,2003-10-28,197,https://i.scdn.co/image/ab67616d00001e020f3572...,55,...,0.6030,0.530000,9,0.0966,-4.919,1,0.0255,104.179,4,0.5390


# Download All Data & Concat

In [20]:
# Download song data
sp_songs_df = get_song_data(all_songs_df)

# Remove rows with duplicate track_id
sp_songs_df = sp_songs_df[~sp_songs_df.duplicated(subset='track_id', keep='first')]

# Save data as csv
sp_songs_df.to_csv('clean_songs.csv',index = False, sep=";")

Song data download complete. Success rate: 57 %
Succesful downloads: 5740 Failed downloads: 4360


In [21]:
# Create list of track_ids from all song data
track_ids = list(sp_songs_df['track_id'])

# Download audio features data
audio_features_df = get_audio_features(track_ids)

# Save data as csv
audio_features_df.to_csv('clean_audio_features.csv',index = False, sep=";")

Audio features download complete. Success rate: 100 %
Succesful downloads: 5688 Failed downloads: 1


In [22]:
# Merge song + audio features data
merge_df = add_audio_features(sp_songs_df, audio_features_df)

# Save data as csv
merge_df.to_csv('clean_songs_and_features.csv',index = False, sep=";")

# Show concat dataset
print(sp_songs_df.shape)
print(audio_features_df.shape)
print(merge_df.shape)

(5689, 11)
(5688, 13)
(5688, 23)
